In [1]:
import os 
from matplotlib import pyplot as plt
import rasterio as ras
import numpy as np
from osgeo import gdal
import os
import pathlib
import rasterio
from rasterio.plot import show
import pandas as pd
import multiprocessing as mp
from itertools import repeat

In [2]:
def ExtractDate(elem):
    return elem[0:8]

In [3]:
#1 Define the WORKDIR
# 请修改i
i=0
sWorkDir = r'C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\'+str(i)
# sWorkDir = r"F:\重庆数据\潼南-高分二号影像\cty\32645"+str(ii)
DataRoot = pathlib.Path(sWorkDir)

##1 search the files with the built-in glob func. of pathlib
lImgfile = list(DataRoot.glob('./*.tif'))
print(f'Total image files: {len(lImgfile)}')

##2 Sorting the file list 
lImgfileName = [img.name for img in lImgfile]
lImgfileName.sort(key=ExtractDate)

print(lImgfileName[2])

Total image files: 7
20240813012330924.tif


In [4]:
#*4 Define a function to determine if a file needs to be extended.
def CheckAndExtend(sImgFileName):
    sSourceImgFile = DataRoot.joinpath(sImgFileName)
    return sSourceImgFile
lImgExtended = list(map(CheckAndExtend,lImgfileName))
lImgEtdName = [img.name for img  in lImgExtended]
print(lImgExtended)

[WindowsPath('C:/zyh/ktywork/20250703sichuan_agriculture/sichuan/time_series/luorun/cty/0/20240311012330924.tif'), WindowsPath('C:/zyh/ktywork/20250703sichuan_agriculture/sichuan/time_series/luorun/cty/0/20240525012330924.tif'), WindowsPath('C:/zyh/ktywork/20250703sichuan_agriculture/sichuan/time_series/luorun/cty/0/20240813012330924.tif'), WindowsPath('C:/zyh/ktywork/20250703sichuan_agriculture/sichuan/time_series/luorun/cty/0/20240823012330924.tif'), WindowsPath('C:/zyh/ktywork/20250703sichuan_agriculture/sichuan/time_series/luorun/cty/0/20240828012330924.tif'), WindowsPath('C:/zyh/ktywork/20250703sichuan_agriculture/sichuan/time_series/luorun/cty/0/20240902012330924.tif'), WindowsPath('C:/zyh/ktywork/20250703sichuan_agriculture/sichuan/time_series/luorun/cty/0/20240912012330924.tif')]


In [5]:
%%time
# 有样本有扩充栅格时
ii=i
#5 将所有影像时序数据读取到内存
##1 定义时序影像立方体变量 data_all
# store the data for all bands of all images including sample and satellite
# data_all[img][band,row,col]
data_all =[]

img_sample = gdal.Open(r'C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\sample\luorun\luorun_sample_prjkc.tif')

if img_sample is None:
    print("打开失败！")
iX_size = img_sample.RasterXSize
iY_size = img_sample.RasterYSize

# img_sample从0, 0)开始，读取一个大小为iX_size乘以iY_size的数组，转换为uint16类型后添加到data_all中
data_all.append(img_sample.ReadAsArray(0,0,iX_size,iY_size).astype('uint16'))
print('行数：{0} 列数：{1}'.format(iY_size,iX_size))

##3 Open all satellite images at once, and read the first 8 bands into the 'data_all'
for i in range(0,len(lImgExtended)):  
    img = gdal.Open(str(lImgExtended[i]))
    print(str(lImgExtended[i]))
    if img is None:
        print("打开失败！")
    img_data_bands = img.ReadAsArray(0,0,iX_size,iY_size).astype('uint16')[3:8,::] #read the 4-8 bands
    data_all.append(img_data_bands)
    img.FlushCache()
    del img
#     print(img_data_bands.shape)
#  因为data_all里面第一个是样本影像 所以下面读取到的会多一个
print('总共打开，并读取到内存的影像数目：{0}'.format(len(data_all)))


行数：924 列数：1233
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0\20240311012330924.tif


<timed exec>:26: RuntimeWarning: invalid value encountered in cast


C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0\20240525012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0\20240813012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0\20240823012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0\20240828012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0\20240902012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0\20240912012330924.tif
总共打开，并读取到内存的影像数目：8
CPU times: total: 6.3 s
Wall time: 6.6 s


In [6]:
%%time 
#%%time,统计这个cell运行需要的时间， ipython提供的魔法函数，必须放在cell的第一行
'''
遍历像素位置，从读取到内存中的影像波段数据中提取时间序列信息
'''
deploy_feature=[] # store time series features of all pixels
sample_feature=[] # store time series features of sample pixels

##4 Extract the time series data pixel by pixel on data_all[img][band,row,col]
for col in range(iX_size):   #xSize
    if col % 50 == 0: #判断处理进度
        print(f'processing line: {col}')
    for row in range(iY_size): #ySize
        ###1 记录该像素的行列号、作物类型编码、隶属地块编号
        sPixelMarkInfor = ','.join([str(ii)+"_"+f'{str(row)}_{str(col)}',
                                   str(data_all[0][0,row,col]),
                                   str(data_all[0][1,row,col])])
        
        ###2 遍历时序影像，提取当下像素位置（row,col）观测日期和波段值
        ####### data_all[img][band,row,col]
        lPixelTimeSeries = [ ','.join([str(lImgEtdName[item[0]-1][0:8]),
                                       str(data_all[item[0]][0,item[1],item[2]]),
                                       str(data_all[item[0]][1,item[1],item[2]]),
                                       str(data_all[item[0]][2,item[1],item[2]]),
#                                        str(data_all[item[0]][3,item[1],item[2]]),
                                       str(data_all[item[0]][4,item[1],item[2]])])
                            for item in zip(range(1,len(lImgEtdName)+1),repeat(row),repeat(col))]  
##                          item：(1, row, col)，(2, row, col)，依此类推
        sPixelTimeSeries = ','.join(lPixelTimeSeries) # transferms a list object to str
        
        ###3 组合每个pixel的标记信息和时间序列波段信息
        sPixelInfor = ','.join([sPixelMarkInfor,sPixelTimeSeries])
        #print (sPixelTimeSeries)
        
        ###4 保存该像素的时间序列信息
        deploy_feature.append(sPixelInfor)
        
        ###5 判断当前pixel是否为样本，是!则同时将其保存到sample_feature
        #### 样本数据的有两个波段，第一波段存放的是样本编码，是四位整数，如果该像素没有样本标注，则为0
        if data_all[0][0,row,col]!=0:
            sample_feature.append(sPixelInfor)
# end looping on each pixel

##5 Write and save the Deploy_feature and sample_feature to the csv file 
with open(DataRoot.joinpath('pixel_deploy.txt'),"w",encoding='utf-8')as f:
#     f.writelines(deploy_feature)
    f.write('\n'.join(deploy_feature)) #write,参数是字符串，不能是list,不符合这里的操作方式

with open(DataRoot.joinpath('pixel_sample.txt'),"w",encoding='utf-8')as f:
#     f.writelines(sample_feature)
    f.write('\n'.join(sample_feature))

print(f'Done!---### please check the path:{str(DataRoot)}')

processing line: 0
processing line: 50
processing line: 100
processing line: 150
processing line: 200
processing line: 250
processing line: 300
processing line: 350
processing line: 400
processing line: 450
processing line: 500
processing line: 550
processing line: 600
processing line: 650
processing line: 700
processing line: 750
processing line: 800
processing line: 850
processing line: 900
processing line: 950
processing line: 1000
processing line: 1050
processing line: 1100
processing line: 1150
processing line: 1200
Done!---### please check the path:C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0
CPU times: total: 1min 12s
Wall time: 1min 15s


In [7]:
# %%time
import os
import pathlib
from shutil import copy
index=[7]    
for i in index:
        sWorkDir = r'C:\zyh\data\fishnet_cty\\'+str(i)
        DataRoot = pathlib.Path(sWorkDir)

        out_path=r"D:\xinjiang\hetian\sample_depoly_txt\\"+str(i)
        if not os.path.exists(out_path):
                os.makedirs(out_path)

        # filepath=r'F:\2018_data\5_Kuitun/32645'+str(ii)
        # rasterfile=str(os.path.join(filepath,os.listdir(filepath)[0]))
        # copy(DataRoot.joinpath('pixel_deploy.txt'),out_path+"/"+'pixel_deploy.txt')
        # copy(rasterfile,out_path+"/"+str(ii)+'.tif')
        copy(DataRoot.joinpath('pixel_sample.txt'),out_path+"/"+'pixel_sample.txt')
print('finish')

finish
